# Tracklock

scrape my deadlock data

discover

look at interesting stats

https://tracklock.gg/players/113272695/matches?page=1

In [13]:
import requests # to scrape the data
from bs4 import BeautifulSoup # to handle the data
import re # for getting wins AND losses 
import pandas as pd # for dataframes
import csv # to create csv file 
import os # to check if file exsists already

### Pull the data:

In [14]:
# get tracklock page respons with requests
# pull data of "player" (current "Ediorian")
playerID = 1676299122
res = requests.get(f"https://tracklock.gg/players/{playerID}/matches?page=1")
res

<Response [200]>

### Read the data with BeautifulSoup

and look at the data

In [15]:
soup = BeautifulSoup(res.text, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1, maximum-scale=1, viewport-fit=cover" name="viewport"/><link as="image" href="/web-app-manifest-512x512.png" rel="preload"/><link data-precedence="next" href="/_next/static/css/b45e69159fbd59aa.css" rel="stylesheet"/><link as="script" fetchpriority="low" href="/_next/static/chunks/webpack-a01c1f682dc3a39c.js" rel="preload"/><script async="" src="/_next/static/chunks/4bd1b696-da1e8c6ae8a01c9a.js"></script><script async="" src="/_next/static/chunks/6215-b1a23dc1c5543a73.js"></script><script async="" src="/_next/static/chunks/main-app-fce99b9b4e4a4ff5.js"></script><script async="" src="/_next/static/chunks/8173-c00e07172c6e21c0.js"></script><script async="" src="/_next/static/chunks/4552-2f902cb32b531943.js"></script><script async="" src="/_next/static/chunks/app/layout-3f9ce2207e041b4d.js"></script><link as="script" href="https://www.googletagmanager.com/gtag/js?id=G-L70ZKKTLC4

In [16]:
games = soup.findAll("tr", class_="transition-colors")

In [17]:
games[-1]

<tr class="transition-colors data-[state=selected]:bg-muted even:bg-[#0F1116] odd:bg-[#11141b] border-b border-[#363636f7] hover:bg-gray-750 text-white"><td class="p-2 align-middle [&amp;:has([role=checkbox])]:pr-0 [&amp;&gt;[role=checkbox]]:translate-y-[2px]"><div class="flex items-center gap-2"><img alt="Yamato" class="w-12 h-12 rounded" data-nimg="1" decoding="async" height="48" loading="lazy" src="/_next/image?url=%2Fpanorama%2Fimages%2Fheroes%2Fyamato_sm_psd.png&amp;w=96&amp;q=75" srcset="/_next/image?url=%2Fpanorama%2Fimages%2Fheroes%2Fyamato_sm_psd.png&amp;w=48&amp;q=75 1x, /_next/image?url=%2Fpanorama%2Fimages%2Fheroes%2Fyamato_sm_psd.png&amp;w=96&amp;q=75 2x" style="color:transparent" width="48"/><div><span class="font-medium">Yamato</span><div class="text-sm text-gray-400">32787891</div></div></div></td><td class="p-2 align-middle [&amp;:has([role=checkbox])]:pr-0 [&amp;&gt;[role=checkbox]]:translate-y-[2px]"><div class="flex flex-col"><span class="text-red-500 font-medium">L

### Find the wins and losses

**Losses:**
tag = span, class "text-red-500 font-medium"

**Wins:**
tag = span, class "text-green-500 font-medium"


In [18]:
# re.compile lets us get both wins and losses, with | -> "or"
winLoss = soup.findAll("span", class_=re.compile(r"text-red-500 font-medium|text-green-500 font-medium")) # works like OR now 
len(winLoss)

20

In [19]:
winLoss[0]

<span class="text-red-500 font-medium">Loss</span>

**save the text in a list:**

In [20]:
winLossData = []
for game in winLoss:
    gameStatus = game.text
    winLossData.append(gameStatus)
winLossData

['Loss',
 'Loss',
 'Win',
 'Loss',
 'Loss',
 'Loss',
 'Loss',
 'Win',
 'Loss',
 'Loss',
 'Loss',
 'Loss',
 'Win',
 'Loss',
 'Loss',
 'Win',
 'Win',
 'Win',
 'Win',
 'Loss']

### Dates: 


**Dates:**
tag = span, class "text-sm text-gray-400"

In [21]:
dates = soup.findAll("span", class_="text-sm text-gray-400")
dates[0]

<span class="text-sm text-gray-400">Feb 14, 2025</span>

**save the dates in a list:**

In [22]:
dateData = []
for game in dates:
    date = game.text
    dateData.append(date)
dateData[:3]

['Feb 14, 2025', 'Feb 14, 2025', 'Feb 14, 2025']

### Duration: 

**Durations:**
tag = div, class = flex flex-col gap-1

this gets us both the match lenggth and date so we do some sketchy shit, every other.

In [23]:
durations = soup.findAll("div", class_="flex flex-col gap-1")
durations[0]

<div class="flex flex-col gap-1"><span>32<!-- --> min</span><svg height="6" width="120"><rect fill="transparent" height="6" rx="3" ry="3" width="120"></rect><rect fill="rgb(128, 150, 186)" height="6" rx="3" ry="3" width="86.14126394052045"></rect></svg></div>

**save the match lengths in a list:**

In [24]:
durationData = []
for i, game in enumerate(durations):
    if (i % 2 == 0): # every other [VERY SKETCHY FIX]
        duration = game.text
        durationData.append(duration)
durationData[:5]

['32 min', '28 min', '26 min', '37 min', '42 min']

### Convert to DataFrame

In [25]:
# traditional loop
rows = [["Win/Loss", "Match length", "Date played"]]# define column names

for i, game in enumerate(winLossData):
    row = [winLossData[i], durationData[i], dateData[i]]
    rows.append(row)
rows

[['Win/Loss', 'Match length', 'Date played'],
 ['Loss', '32 min', 'Feb 14, 2025'],
 ['Loss', '28 min', 'Feb 14, 2025'],
 ['Win', '26 min', 'Feb 14, 2025'],
 ['Loss', '37 min', 'Feb 12, 2025'],
 ['Loss', '42 min', 'Feb 12, 2025'],
 ['Loss', '42 min', 'Feb 12, 2025'],
 ['Loss', '31 min', 'Feb 12, 2025'],
 ['Win', '35 min', 'Feb 12, 2025'],
 ['Loss', '29 min', 'Feb 12, 2025'],
 ['Loss', '41 min', 'Feb 12, 2025'],
 ['Loss', '20 min', 'Feb 12, 2025'],
 ['Loss', '33 min', 'Feb 12, 2025'],
 ['Win', '28 min', 'Feb 12, 2025'],
 ['Loss', '31 min', 'Feb 12, 2025'],
 ['Loss', '29 min', 'Feb 12, 2025'],
 ['Win', '42 min', 'Feb 12, 2025'],
 ['Win', '39 min', 'Feb 10, 2025'],
 ['Win', '27 min', 'Feb 10, 2025'],
 ['Win', '21 min', 'Feb 10, 2025'],
 ['Loss', '42 min', 'Feb 10, 2025']]

In [26]:
# create the pandas dataframe 
df = pd.DataFrame(rows)
df.head(3)

,0,1,2
0,Win/Loss,Match length,Date played
1,Loss,32 min,"Feb 14, 2025"
2,Loss,28 min,"Feb 14, 2025"


In [27]:
# make it prettier
df.columns = df.iloc[0] # first row as column names
df = df.drop(0).reset_index(drop=True) # remove the row and reset index
df

,Win/Loss,Match length,Date played
0,Loss,32 min,"Feb 14, 2025"
1,Loss,28 min,"Feb 14, 2025"
2,Win,26 min,"Feb 14, 2025"
3,Loss,37 min,"Feb 12, 2025"
4,Loss,42 min,"Feb 12, 2025"
5,Loss,42 min,"Feb 12, 2025"
6,Loss,31 min,"Feb 12, 2025"
7,Win,35 min,"Feb 12, 2025"
8,Loss,29 min,"Feb 12, 2025"
9,Loss,41 min,"Feb 12, 2025"


In [28]:
# describe, shape, type
print("describe:\n", df.describe(), end="\n\n")
print("shape:\n", df.shape, end="\n\n")
print("type:\n", type(df))

describe:
 0      Win/Loss Match length   Date played
count        20           20            20
unique        2           14             3
top        Loss       42 min  Feb 12, 2025
freq         13            4            13

shape:
 (20, 3)

type:
 <class 'pandas.core.frame.DataFrame'>


In [29]:
print(df["Match length"].max())
print(df["Match length"].min())

42 min
20 min


### Export as CSV:

In [ ]:
# create CSV file out of dataframe
if (not os.path.exists(f'playerID{playerID}_dl_stats.csv')): #create new csv
    df.to_csv(f'playerID{playerID}_dl_stats.csv', index=False)
    print("new csv file created.")
else: 
    print("File already exists.")
    #want to append?
    #df.to_csv(f'playerID{playerID}_dl_stats.csv', mode='a', index=False, header=False) # we dont want to rewrite column titles
    #print("appended to csv file")

new csv file created


### Functions

- in order to get many pages
- easily control the pulled data

In [31]:
# pull the data of all available pages and return them as a list of pages
def pullTracklockPages(playerId):
    """pull all available data for given player. Return list of all pages."""
    # get tracklock page respons with requests 
    res = requests.get(f"https://tracklock.gg/players/{playerId}/matches?page=1")
    soup = BeautifulSoup(res.text, 'html.parser')
    
    # get total pages number
    pageNumbers = soup.findAll("li")
    totalPageNumber = int(pageNumbers[-2].text)
    print("total pages to import:", totalPageNumber)
    
    # get all pages
    allPages = []
    for pageNumber in range(1, totalPageNumber + 1):
        res = requests.get(f"https://tracklock.gg/players/{playerId}/matches?page={pageNumber}")
        
        print("Page", pageNumber, "was pulled.")
        
        soup = BeautifulSoup(res.text, 'html.parser')
        allPages.append(soup)
    return allPages

In [45]:
# pull data for "Edorian" and save as variable
allPagesData = pullTracklockPages(playerID) 

total pages to import: 79
Page 1 was pulled.
Page 2 was pulled.
Page 3 was pulled.
Page 4 was pulled.
Page 5 was pulled.
Page 6 was pulled.
Page 7 was pulled.
Page 8 was pulled.
Page 9 was pulled.
Page 10 was pulled.
Page 11 was pulled.
Page 12 was pulled.
Page 13 was pulled.
Page 14 was pulled.
Page 15 was pulled.
Page 16 was pulled.
Page 17 was pulled.
Page 18 was pulled.
Page 19 was pulled.
Page 20 was pulled.
Page 21 was pulled.
Page 22 was pulled.
Page 23 was pulled.
Page 24 was pulled.
Page 25 was pulled.
Page 26 was pulled.
Page 27 was pulled.
Page 28 was pulled.
Page 29 was pulled.
Page 30 was pulled.
Page 31 was pulled.
Page 32 was pulled.
Page 33 was pulled.
Page 34 was pulled.
Page 35 was pulled.
Page 36 was pulled.
Page 37 was pulled.
Page 38 was pulled.
Page 39 was pulled.
Page 40 was pulled.
Page 41 was pulled.
Page 42 was pulled.
Page 43 was pulled.
Page 44 was pulled.
Page 45 was pulled.
Page 46 was pulled.
Page 47 was pulled.
Page 48 was pulled.
Page 49 was pulled.
Pag

In [47]:
len(allPagesData)

79

In [58]:
# function for getting the data from the pages
def scrapeTracklock(pages):
    """Scrape the data of of the pages that are given. Return """

    winLossData = []
    dateData = []
    durationData = []

    for page in pages:
        # Extract Win/Loss data
        winLoss = page.findAll("span", class_=re.compile(r"text-red-500 font-medium|text-green-500 font-medium"))
        winLossData.extend(game.text for game in winLoss)

        # Extract Dates
        dates = page.findAll("span", class_="text-sm text-gray-400")
        dateData.extend(date.text for date in dates)

        # Extract Match Durations
        durations = page.findAll("div", class_="flex flex-col gap-1")
        for i, game in enumerate(durations):
            if i % 2 == 0:  # Every other one
                durationData.append(game.text)

    
    return winLossData, dateData, durationData

In [59]:
# use scrapeTacklock() on gotten data
scrapeTracklock(allPagesData)

(['Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'W

In [60]:
fullData = scrapeTracklock(allPagesData)
fullData

(['Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Win',
  'Loss',
  'Win',
  'Win',
  'Loss',
  'Loss',
  'Loss',
  'Loss',
  'Win',
  'W

### Export FULL data as CSV

In [61]:
# data into a pd.DataFrame
df = pd.DataFrame({
    'Result': fullData[0],
    'Date': fullData[1],
    'Duration': fullData[2]
})

print(df)

     Result          Date Duration
0      Loss  Feb 14, 2025   32 min
1      Loss  Feb 14, 2025   28 min
2       Win  Feb 14, 2025   26 min
3      Loss  Feb 12, 2025   37 min
4      Loss  Feb 12, 2025   42 min
...     ...           ...      ...
1572   Loss  Jun 20, 2024   22 min
1573    Win  Jun 20, 2024   32 min
1574   Loss  Jun 20, 2024   38 min
1575   Loss  Jun 20, 2024   30 min
1576    Win  Jun 20, 2024   31 min

[1577 rows x 3 columns]


In [62]:
# create CSV file out of dataframe
if (not os.path.exists(f'playerID{playerID}_dl_stats.csv')): #create new csv
    df.to_csv(f'playerID{playerID}_dl_stats.csv', index=False)
    print("new csv file created.")
else: 
    print("File already exists.")

new csv file created.


### CONTINUE:

add to scrapeTracklock() more data pulle